## Manual Data Ingestion

In [1]:
# Vector Embeddings & Similarity Foundation
import numpy as np
from sentence_transformers import SentenceTransformer
import asyncio
from datetime import datetime
from typing import List, Tuple, Dict, Any
from dataclasses import dataclass
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("AGENT MEMORY & CONVERSATION MANAGEMENT")
print("=" * 45)
print(f"Session: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Focus: Vector embeddings and similarity search")
print()

# Initialize embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

def create_embedding(text: str) -> np.ndarray:
    """Create normalized embedding for text"""
    embedding = embedding_model.encode([text])[0]
    # Normalize for cosine similarity
    return embedding / np.linalg.norm(embedding)

def calculate_similarity(embedding1: np.ndarray, embedding2: np.ndarray) -> float:
    """Calculate cosine similarity between embeddings"""
    return float(np.dot(embedding1, embedding2))

# Test embedding functionality
print("🔍 Testing Vector Embeddings:")
test_texts = [
    "Lingkang kuli kuli wachang lingkangku lingkangku",
    "Hidup Blonde!", 
    "Yo ndak tau kok tanya saya",
    "Kamu siapa? Aku siapa?"
]

embeddings = []
for text in test_texts:
    embedding = create_embedding(text)
    embeddings.append((text, embedding))
    print(f"Created embedding for: '{text}' (dimension: {len(embedding)})")

# Test similarity
print("\nSimilarity Analysis:")
for i in range(len(embeddings)):
    for j in range(i+1, len(embeddings)):
        text1, emb1 = embeddings[i]
        text2, emb2 = embeddings[j]
        similarity = calculate_similarity(emb1, emb2)
        print(f"   '{text1[:30]}...' vs '{text2[:30]}...': {similarity:.3f}")

print("\nVector embedding system working:")
print("Sentence transformer model loaded")
print("Embedding dimension: {len(embeddings[0][1])}")
print("Cosine similarity calculation ready")

d:\Coding Task\Google-ADK\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


AGENT MEMORY & CONVERSATION MANAGEMENT
Session: 2025-07-27 22:02:52
Focus: Vector embeddings and similarity search



Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
d:\Coding Task\Google-ADK\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adity\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see

🔍 Testing Vector Embeddings:


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.86it/s]


...

Created embedding for: 'Hidup Blonde!' (dimension: 384)


Batches: 100%|██████████| 1/1 [00:00<00:00, 24.89it/s]


Created embedding for: 'Yo ndak tau kok tanya saya' (dimension: 384)


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.32it/s]

Created embedding for: 'Kamu siapa? Aku siapa?' (dimension: 384)

Similarity Analysis:
   'Lingkang kuli kuli wachang lin...' vs 'Hidup Blonde!...': 0.167
   'Lingkang kuli kuli wachang lin...' vs 'Yo ndak tau kok tanya saya...': 0.348
   'Lingkang kuli kuli wachang lin...' vs 'Kamu siapa? Aku siapa?...': 0.429
   'Hidup Blonde!...' vs 'Yo ndak tau kok tanya saya...': 0.289
   'Hidup Blonde!...' vs 'Kamu siapa? Aku siapa?...': 0.200
   'Yo ndak tau kok tanya saya...' vs 'Kamu siapa? Aku siapa?...': 0.342

Vector embedding system working:
Sentence transformer model loaded
Embedding dimension: {len(embeddings[0][1])}
Cosine similarity calculation ready


## Data Ingestion using FAISS

In [2]:
# FAISS Integration for Vector Search
import faiss
import uuid
import json

@dataclass
class MemoryEntry:
    """Structured memory entry with metadata"""
    entry_id: str
    content: str
    embedding: np.ndarray
    metadata: Dict[str, Any]
    created_at: str

class VectorMemorySystem:
    """Production vector memory with FAISS"""
    
    def __init__(self, dimension: int = 384):
        self.dimension = dimension
        self.index = faiss.IndexFlatIP(dimension)  # Inner product for cosine similarity
        self.entries = []
        self.entry_map = {}  # Maps FAISS index to entry_id
        
    def add_memory(self, content: str, metadata: Dict[str, Any] = None) -> str:
        """Add content to vector memory"""
        if metadata is None:
            metadata = {}
            
        entry_id = str(uuid.uuid4())
        embedding = create_embedding(content)
        
        # Create memory entry
        entry = MemoryEntry(
            entry_id=entry_id,
            content=content,
            embedding=embedding,
            metadata=metadata,
            created_at=datetime.now().isoformat()
        )
        
        # Add to FAISS index
        faiss_index = len(self.entries)
        self.index.add(embedding.reshape(1, -1))
        self.entries.append(entry)
        self.entry_map[faiss_index] = entry_id
        
        logger.info(f"Added memory: {entry_id[:8]} - '{content[:50]}...'")
        return entry_id
    
    def search_memory(self, query: str, top_k: int = 5, threshold: float = 0.3) -> List[Tuple[MemoryEntry, float]]:
        """Search memory using vector similarity"""
        if len(self.entries) == 0:
            return []
        
        query_embedding = create_embedding(query)
        scores, indices = self.index.search(query_embedding.reshape(1, -1), min(top_k, len(self.entries)))
        
        results = []
        for score, idx in zip(scores[0], indices[0]):
            if score >= threshold:
                entry = self.entries[idx]
                results.append((entry, float(score)))
        
        logger.info(f"Memory search for '{query[:30]}...' found {len(results)} matches")
        return results
    
    def get_stats(self) -> Dict[str, Any]:
        """Get memory statistics"""
        return {
            "total_entries": len(self.entries),
            "index_size": self.index.ntotal,
            "dimension": self.dimension
        }

# Initialize vector memory
vector_memory = VectorMemorySystem()

# Test with knowledge entries
print("\n🔧 Testing FAISS Vector Memory:")
knowledge_base = [
    ("Python is a programming language known for simplicity and readability", {"topic": "programming"}),
    ("Machine learning enables computers to learn from data automatically", {"topic": "ai"}),
    ("FAISS provides efficient similarity search for large vector collections", {"topic": "search"}),
    ("Vector databases store high-dimensional embeddings for semantic search", {"topic": "databases"}),
    ("Natural language processing helps computers understand human language", {"topic": "nlp"})
]

for content, metadata in knowledge_base:
    entry_id = vector_memory.add_memory(content, metadata)

# Test search functionality
print("\nTesting Vector Search:")
test_queries = [
    "programming languages",
    "artificial intelligence and learning", 
    "search algorithms"
]

for query in test_queries:
    print(f"\nQuery: '{query}'")
    results = vector_memory.search_memory(query, top_k=2)
    for entry, score in results:
        print(f"Match (score: {score:.3f}): {entry.content}")

print("\nFAISS vector memory system ready:")
stats = vector_memory.get_stats()
print("Total entries: {stats['total_entries']}")
print("Vector dimension: {stats['dimension']}")
print("FAISS index size: {stats['index_size']}")

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:faiss:Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.



🔧 Testing FAISS Vector Memory:


Batches: 100%|██████████| 1/1 [00:00<00:00,  8.21it/s]
INFO:__main__:Added memory: 00ae0d3f - 'Python is a programming language known for simplic...'
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.40it/s]
INFO:__main__:Added memory: 5f97ecba - 'Machine learning enables computers to learn from d...'
Batches: 100%|██████████| 1/1 [00:00<00:00,  2.55it/s]
INFO:__main__:Added memory: fcd91dfe - 'FAISS provides efficient similarity search for lar...'
Batches: 100%|██████████| 1/1 [00:00<00:00,  8.27it/s]
INFO:__main__:Added memory: 31c1f43d - 'Vector databases store high-dimensional embeddings...'
Batches: 100%|██████████| 1/1 [00:00<00:00, 11.92it/s]
INFO:__main__:Added memory: 70e2f912 - 'Natural language processing helps computers unders...'



Testing Vector Search:

Query: 'programming languages'


Batches: 100%|██████████| 1/1 [00:00<00:00, 29.16it/s]
INFO:__main__:Memory search for 'programming languages...' found 2 matches


Match (score: 0.579): Python is a programming language known for simplicity and readability
Match (score: 0.401): Natural language processing helps computers understand human language

Query: 'artificial intelligence and learning'


Batches: 100%|██████████| 1/1 [00:00<00:00, 35.63it/s]
INFO:__main__:Memory search for 'artificial intelligence and le...' found 2 matches


Match (score: 0.522): Machine learning enables computers to learn from data automatically
Match (score: 0.359): Natural language processing helps computers understand human language

Query: 'search algorithms'


Batches: 100%|██████████| 1/1 [00:00<00:00, 38.03it/s]
INFO:__main__:Memory search for 'search algorithms...' found 2 matches


Match (score: 0.457): FAISS provides efficient similarity search for large vector collections
Match (score: 0.401): Vector databases store high-dimensional embeddings for semantic search

FAISS vector memory system ready:
Total entries: {stats['total_entries']}
Vector dimension: {stats['dimension']}
FAISS index size: {stats['index_size']}


## Memory Conversation Management using SQlite

In [3]:
# Conversation Persistence Manager
import sqlite3
from pathlib import Path

@dataclass
class ConversationTurn:
    """Individual conversation turn with metadata"""
    turn_id: str
    session_id: str
    timestamp: str
    user_message: str
    agent_response: str
    knowledge_used: List[str]

class ConversationManager:
    """Manages conversation persistence with SQLite"""
    
    def __init__(self, db_path: str = "agent_memory.db"):
        self.db_path = db_path
        self.init_database()
        
    def init_database(self):
        """Initialize SQLite database"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Sessions table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS sessions (
                session_id TEXT PRIMARY KEY,
                user_id TEXT NOT NULL,
                started_at TEXT NOT NULL,
                last_active TEXT NOT NULL,
                total_turns INTEGER DEFAULT 0
            )
        ''')
        
        # Conversation turns table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS turns (
                turn_id TEXT PRIMARY KEY,
                session_id TEXT NOT NULL,
                timestamp TEXT NOT NULL,
                user_message TEXT NOT NULL,
                agent_response TEXT NOT NULL,
                knowledge_used TEXT,
                FOREIGN KEY (session_id) REFERENCES sessions (session_id)
            )
        ''')
        
        conn.commit()
        conn.close()
        logger.info(f"Database initialized: {self.db_path}")
    
    def create_session(self, user_id: str) -> str:
        """Create new conversation session"""
        session_id = str(uuid.uuid4())
        timestamp = datetime.now().isoformat()
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT INTO sessions (session_id, user_id, started_at, last_active, total_turns)
            VALUES (?, ?, ?, ?, ?)
        ''', (session_id, user_id, timestamp, timestamp, 0))
        
        conn.commit()
        conn.close()
        
        logger.info(f"Created session: {session_id[:8]} for user {user_id}")
        return session_id
    
    def save_turn(self, session_id: str, user_message: str, agent_response: str, 
                  knowledge_used: List[str] = None) -> str:
        """Save conversation turn"""
        turn_id = str(uuid.uuid4())
        timestamp = datetime.now().isoformat()
        
        if knowledge_used is None:
            knowledge_used = []
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Save turn
        cursor.execute('''
            INSERT INTO turns (turn_id, session_id, timestamp, user_message, agent_response, knowledge_used)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (turn_id, session_id, timestamp, user_message, agent_response, json.dumps(knowledge_used)))
        
        # Update session
        cursor.execute('''
            UPDATE sessions SET last_active = ?, total_turns = total_turns + 1
            WHERE session_id = ?
        ''', (timestamp, session_id))
        
        conn.commit()
        conn.close()
        
        logger.info(f"Saved turn: {turn_id[:8]} for session {session_id[:8]}")
        return turn_id
    
    def get_conversation_history(self, session_id: str, last_n: int = 5) -> List[ConversationTurn]:
        """Get recent conversation history"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            SELECT turn_id, session_id, timestamp, user_message, agent_response, knowledge_used
            FROM turns WHERE session_id = ?
            ORDER BY timestamp DESC LIMIT ?
        ''', (session_id, last_n))
        
        turns = []
        for row in cursor.fetchall():
            turn = ConversationTurn(
                turn_id=row[0],
                session_id=row[1], 
                timestamp=row[2],
                user_message=row[3],
                agent_response=row[4],
                knowledge_used=json.loads(row[5]) if row[5] else []
            )
            turns.append(turn)
        
        conn.close()
        turns.reverse()  # Chronological order
        
        logger.info(f"Retrieved {len(turns)} turns for session {session_id[:8]}")
        return turns

# Initialize conversation manager
conversation_manager = ConversationManager()

# Test conversation persistence
print("\nTesting Conversation Persistence:")
test_session = conversation_manager.create_session("demo_user")

# Save some test conversations
test_conversations = [
    ("Hello, I'm learning about AI", "Hi! I'd be happy to help you learn about AI. What specifically interests you?"),
    ("What is machine learning?", "Machine learning is a subset of AI that enables computers to learn from data without explicit programming."),
    ("How does it relate to neural networks?", "Neural networks are one approach to machine learning, inspired by how biological brains process information.")
]

for user_msg, agent_resp in test_conversations:
    turn_id = conversation_manager.save_turn(test_session, user_msg, agent_resp, ["ai_knowledge"])

# Test retrieval
print("\nTesting History Retrieval:")
history = conversation_manager.get_conversation_history(test_session, last_n=3)
for i, turn in enumerate(history, 1):
    print(f"   Turn {i}:")
    print(f"     User: {turn.user_message}")
    print(f"     Agent: {turn.agent_response}")

print("\nConversation persistence ready:")
print("SQLite database: {conversation_manager.db_path}")
print("Session tracking with turn history")
print("Knowledge usage logging")

INFO:__main__:Database initialized: agent_memory.db
INFO:__main__:Created session: f884f9e9 for user demo_user
INFO:__main__:Saved turn: 26501cf3 for session f884f9e9
INFO:__main__:Saved turn: eef9b93b for session f884f9e9
INFO:__main__:Saved turn: 850d8eac for session f884f9e9
INFO:__main__:Retrieved 3 turns for session f884f9e9



Testing Conversation Persistence:

Testing History Retrieval:
   Turn 1:
     User: Hello, I'm learning about AI
     Agent: Hi! I'd be happy to help you learn about AI. What specifically interests you?
   Turn 2:
     User: What is machine learning?
     Agent: Machine learning is a subset of AI that enables computers to learn from data without explicit programming.
   Turn 3:
     User: How does it relate to neural networks?
     Agent: Neural networks are one approach to machine learning, inspired by how biological brains process information.

Conversation persistence ready:
SQLite database: {conversation_manager.db_path}
Session tracking with turn history
Knowledge usage logging


## Agent Integration

In [ ]:
# Memory-Enhanced Agent with ADK (Fixed Session Management)
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.models.lite_llm import LiteLlm
from google.genai import types

class MemoryAgent:
    """Agent with vector memory and conversation persistence"""
    
    def __init__(self):
        self.vector_memory = vector_memory
        self.conversation_manager = conversation_manager
        self.current_session = None
        self.adk_session_id = None  # Separate ADK session ID
        
    async def setup(self, user_id: str = "demo_user"):
        """Initialize memory-enhanced agent"""
        # Create conversation session
        self.current_session = self.conversation_manager.create_session(user_id)
        
        # Create separate ADK session ID
        self.adk_session_id = f"adk_{self.current_session}"
        
        # Setup ADK agent
        model = LiteLlm(model="ollama_chat/llama3.1:8b")
        
        self.agent = Agent(
            name="MemoryAgent",
            model=model,
            instruction="""You are an intelligent agent with persistent memory capabilities.

            You have access to:
            - A knowledge base with relevant information
            - Previous conversation history with this user
            - Context from past interactions

            When responding:
            1. Use relevant knowledge from your memory when helpful
            2. Reference previous conversations naturally
            3. Build on past context to provide personalized responses
            4. Be conversational and remember what you've discussed

            Always be helpful and maintain conversation continuity."""
        )
        
        self.session_service = InMemorySessionService()
        self.runner = Runner(
            agent=self.agent,
            app_name="memory_agent", 
            session_service=self.session_service
        )
        
        # Create ADK session with the correct ID
        await self.session_service.create_session(
            app_name="memory_agent",
            user_id=user_id,
            session_id=self.adk_session_id
        )
        
        logger.info(f"Memory agent ready - Conversation: {self.current_session[:8]}, ADK: {self.adk_session_id[:8]}")
    
    async def chat(self, user_message: str) -> str:
        """Chat with memory-enhanced responses"""
        # Search relevant knowledge
        knowledge_results = self.vector_memory.search_memory(user_message, top_k=3, threshold=0.2)
        relevant_knowledge = [entry.content for entry, score in knowledge_results]
        
        # Get conversation history
        history = self.conversation_manager.get_conversation_history(self.current_session, last_n=3)
        
        # Build context
        context_parts = []
        
        if relevant_knowledge:
            context_parts.append("Relevant Knowledge:")
            for knowledge in relevant_knowledge:
                context_parts.append(f"- {knowledge}")
        
        if history:
            context_parts.append("\nRecent Conversation:")
            for turn in history:
                context_parts.append(f"User: {turn.user_message}")
                context_parts.append(f"Assistant: {turn.agent_response}")
        
        context_parts.append(f"\nCurrent User Message: {user_message}")
        
        enhanced_message = "\n".join(context_parts)
        
        # Send to agent using the correct ADK session ID
        message = types.Content(role="user", parts=[types.Part(text=enhanced_message)])
        
        response = ""
        async for event in self.runner.run_async(
            user_id="memory_user",
            session_id=self.adk_session_id,  # Use ADK session ID here
            new_message=message
        ):
            if event.is_final_response():
                response = event.content.parts[0].text
                break
        
        # Save conversation using conversation session ID
        knowledge_used = [f"{entry.content[:50]}..." for entry, _ in knowledge_results]
        self.conversation_manager.save_turn(self.current_session, user_message, response, knowledge_used)
        
        return response

# Initialize memory agent
memory_agent = MemoryAgent()
await memory_agent.setup("demo_user")

print("Memory-enhanced agent ready:")
print("Vector knowledge retrieval integrated")
print("Conversation history tracking")
print("Context-aware response generation")
print("Session management fixed")

INFO:google_adk.google.adk.models.registry:Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for model-optimizer-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/endpoints\/.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/publishers\/google\/models\/gemini.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google

Memory-enhanced agent ready:
Vector knowledge retrieval integrated
Conversation history tracking
Context-aware response generation
Session management fixed


## Agent Execution

In [6]:
# Complete Memory System Demonstration
async def chat(self, user_message: str) -> str:
    """Chat with memory-enhanced responses"""
    # Search relevant knowledge
    knowledge_results = self.vector_memory.search_memory(user_message, top_k=3, threshold=0.2)
    relevant_knowledge = [entry.content for entry, score in knowledge_results]
    
    # Get conversation history
    history = self.conversation_manager.get_conversation_history(self.current_session, last_n=3)
    
    # Build context
    context_parts = []
    
    if relevant_knowledge:
        context_parts.append("Relevant Knowledge:")
        for knowledge in relevant_knowledge:
            context_parts.append(f"- {knowledge}")
    
    if history:
        context_parts.append("\nRecent Conversation:")
        for turn in history:
            context_parts.append(f"User: {turn.user_message}")
            context_parts.append(f"Assistant: {turn.agent_response}")
    
    context_parts.append(f"\nCurrent User Message: {user_message}")
    
    enhanced_message = "\n".join(context_parts)
    
    # Send to agent using the correct user_id
    message = types.Content(role="user", parts=[types.Part(text=enhanced_message)])
    
    response = ""
    async for event in self.runner.run_async(
        user_id="demo_user",  # Fixed: use same user_id as setup
        session_id=self.adk_session_id,
        new_message=message
    ):
        if event.is_final_response():
            response = event.content.parts[0].text
            break
    
    # Save conversation
    knowledge_used = [f"{entry.content[:50]}..." for entry, _ in knowledge_results]
    self.conversation_manager.save_turn(self.current_session, user_message, response, knowledge_used)
    
    return response

# Update the method properly
MemoryAgent.chat = chat

async def demonstrate_memory_system():
    """Test memory-enhanced conversations"""
    
    print("MEMORY SYSTEM DEMONSTRATION")
    print("=" * 35)
    
    conversations = [
        "Hi, I'm interested in learning about Python programming",
        "What makes Python good for machine learning?", 
        "Can you explain how vector search works?",
        "Earlier you mentioned Python - what did you say about its advantages?",
        "How do vector databases help with the machine learning concepts we discussed?"
    ]
    
    print("\n Memory-Enhanced Conversations:")
    
    for i, user_message in enumerate(conversations, 1):
        print(f"\n--- Turn {i} ---")
        print(f"👤 User: {user_message}")
        
        response = await memory_agent.chat(user_message)
        print(f"Agent: {response}")
        
        await asyncio.sleep(0.5)

# Run demonstration
await demonstrate_memory_system()

print("\nMEMORY SYSTEM ANALYSIS")
print("=" * 30)

# Memory statistics
memory_stats = vector_memory.get_stats()
print("\nVector Memory:")
print(f"Knowledge Entries: {memory_stats['total_entries']}")
print(f"Vector Dimension: {memory_stats['dimension']}")
print(f"FAISS Index Size: {memory_stats['index_size']}")

# Test knowledge retrieval
print("\nKnowledge Retrieval Test:")
test_query = "programming and artificial intelligence"
results = vector_memory.search_memory(test_query, top_k=3)
print(f"Query: '{test_query}'")
for entry, score in results:
    print(f"✅ {score:.3f}: {entry.content[:60]}...")

# Conversation history
history = conversation_manager.get_conversation_history(memory_agent.current_session)
print("\nConversation History:")
print(f"Total turns in session: {len(history)}")
print(f"Session ID: {memory_agent.current_session[:8]}")

MEMORY SYSTEM DEMONSTRATION

 Memory-Enhanced Conversations:

--- Turn 1 ---
👤 User: Hi, I'm interested in learning about Python programming


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.91it/s]
INFO:__main__:Memory search for 'Hi, I'm interested in learning...' found 2 matches
INFO:__main__:Retrieved 0 turns for session defb9018
22:10:45 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= llama3.1:8b; provider = ollama_chat
INFO:LiteLLM:
LiteLLM completion() model= llama3.1:8b; provider = ollama_chat
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:__main__:Saved turn: db9654e8 for session defb9018


Agent: Hello! It's great to hear that you're interested in learning about Python programming. We've discussed languages before, but it seems we're starting fresh this time.

Python is indeed a fantastic choice for beginners and experienced programmers alike due to its simplicity and readability. I'd be happy to help you get started with some beginner-friendly resources. Would you like me to suggest some tutorials or online courses that can help you learn the basics of Python?

Also, what specifically are you hoping to do with Python programming? Are you interested in web development, data analysis, artificial intelligence, or something else? Knowing your goals will help me provide more targeted guidance.


INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"



--- Turn 2 ---
👤 User: What makes Python good for machine learning?


...

Agent: You're connecting the dots between vector databases and machine learning. That's a great question!

Vector databases, such as those using FAISS for efficient similarity search, can greatly enhance the machine learning workflow by providing an efficient way to store and query high-dimensional embeddings.

Think of it this way: in machine learning, you often have large datasets with complex relationships between data points. Vector databases help you represent these relationships in a compact and computationally efficient manner. This enables fast lookup times for similar vectors, which is particularly useful when working with text-based data, such as natural language processing tasks.

By integrating vector databases into your machine learning pipeline, you can:

1. **Efficiently store** high-dimensional embeddings for large datasets.
2. **Speed up similarity searches**, allowing you to quickly identify relevant patterns or relationships in the data.
3. **Improve model performanc

INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"



MEMORY SYSTEM ANALYSIS

Vector Memory:
Knowledge Entries: 5
Vector Dimension: 384
FAISS Index Size: 5

Knowledge Retrieval Test:


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.54it/s]
INFO:__main__:Memory search for 'programming and artificial int...' found 3 matches
INFO:__main__:Retrieved 5 turns for session defb9018


Query: 'programming and artificial intelligence'
✅ 0.468: Python is a programming language known for simplicity and re...
✅ 0.431: Machine learning enables computers to learn from data automa...
✅ 0.425: Natural language processing helps computers understand human...

Conversation History:
Total turns in session: 5
Session ID: defb9018
